<a href="https://colab.research.google.com/github/billgatos/IA25_P01_G03/blob/teste-com-matplotlib/trab1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install contraint library
!pip install python-constraint

from constraint import *
import random

# import constraint
# from constraint import *
# import random

# --- 1. DATA DEFINITION AND CONSTANTS ---

# Time domain constants
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
BLOCKS = list(range(1, 21))  # 1 to 20 total blocks (5 days * 4 blocks/day)
# Example: Block 1 = Monday S1, Block 5 = Tuesday S1, Block 20 = Friday S4
ROOMS = ['R1', 'R2', 'R3', 'R4', 'Lab01', 'Online'] # A set of general and specific rooms

# --- Input Data Mapping ---

# Class to Courses (cc)
CLASS_COURSES = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35'],
}
# Assumption: All course-class pairs have 2 lessons per week.

# Course to Teacher (dsd) - Inverted for easy lookup
COURSE_TEACHERS = {}
TEACHER_COURSES = {
    'jo': ['UC11', 'UC21', 'UC22', 'UC31'],
    'mike': ['UC12', 'UC23', 'UC32'],
    'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
    'sue': ['UC15', 'UC25', 'UC34', 'UC35'],
}
for teacher, courses in TEACHER_COURSES.items():
    for course in courses:
        COURSE_TEACHERS[course] = teacher

# Teacher Time Restrictions (tr)
TEACHER_UNAVAILABLE_SLOTS = {
    'mike': [13, 14, 15, 16, 17, 18, 19, 20],
    'rob': [1, 2, 3, 4],
    'sue': [9, 10, 11, 12, 17, 18, 19, 20],
}

# Room Restrictions (rr)
COURSE_ROOM_RESTRICTIONS = {
    'UC14': 'Lab01',
    'UC22': 'Lab01',
}

# Online Classes (oc) - Lesson 2 of these courses must be online
ONLINE_LESSONS = {
    'UC21': 2,
    'UC31': 2,
}

# --- 2. VARIABLE GENERATION ---

# Variables are of the form 'Class_Course_LessonIndex' (e.g., 't01_UC11_L1')
LESSON_VARIABLES = []
for class_id, courses in CLASS_COURSES.items():
    for course in courses:
        # Each course-class pair has 2 lessons
        LESSON_VARIABLES.append(f'{class_id}_{course}_L1')
        LESSON_VARIABLES.append(f'{class_id}_{course}_L2')

# --- 3. DOMAIN GENERATION (TIME + ROOM) ---

# The Universal Domain: All possible (Time_Block, Room) combinations
UNIVERSAL_DOMAIN = []
for block in BLOCKS:
    for room in ROOMS:
        UNIVERSAL_DOMAIN.append((block, room))

# --- 4. CSP MODEL INITIALIZATION ---

problem = Problem()

# Helper function to get information from the variable string
def get_lesson_info(variable_name):
    parts = variable_name.split('_')
    class_id = parts[0]
    course = parts[1]
    lesson_index = int(parts[2][1:]) # L1 -> 1, L2 -> 2
    teacher = COURSE_TEACHERS.get(course)
    return class_id, course, lesson_index, teacher

# 4a. Add Variables with Restricted Domains

for var in LESSON_VARIABLES:
    class_id, course, lesson_index, teacher = get_lesson_info(var)

    # Start with the full domain
    restricted_domain = list(UNIVERSAL_DOMAIN)

    # 1. Apply Teacher Time Restrictions (tr)
    if teacher in TEACHER_UNAVAILABLE_SLOTS:
        unavailable_slots = TEACHER_UNAVAILABLE_SLOTS[teacher]
        # Remove (Block, Room) pairs where Block is unavailable
        restricted_domain = [
            (block, room) for block, room in restricted_domain
            if block not in unavailable_slots
        ]

    # 2. Apply Room Restrictions (rr)
    required_room = COURSE_ROOM_RESTRICTIONS.get(course)
    if required_room:
        # Filter to keep only the required room
        restricted_domain = [
            (block, room) for block, room in restricted_domain
            if room == required_room
        ]

    # 3. Apply Online Class Constraint (oc)
    if lesson_index == ONLINE_LESSONS.get(course):
        # Filter to keep only the 'Online' room
        restricted_domain = [
            (block, room) for block, room in restricted_domain
            if room == 'Online'
        ]

    problem.addVariable(var, restricted_domain)

# --- 5. HARD CONSTRAINT IMPLEMENTATION ---

# Constraint A: Class Conflicts (A class can only have one lesson per block)
# Every class's lessons must be in unique (Block) combinations.
for class_id in CLASS_COURSES.keys():
    class_lessons = [var for var in LESSON_VARIABLES if var.startswith(f'{class_id}_')]

    # For every pair of lessons in the same class
    for i in range(len(class_lessons)):
        for j in range(i + 1, len(class_lessons)):
            var_a = class_lessons[i]
            var_b = class_lessons[j]

            # Constraint: Block of Lesson A != Block of Lesson B
            # Assignment is (Block, Room)
            problem.addConstraint(lambda a, b: a[0] != b[0], (var_a, var_b))

# Constraint B: Teacher Conflicts (A teacher can only teach one lesson per block)
# All lessons taught by the same teacher must be in unique (Block) combinations.
for teacher, courses in TEACHER_COURSES.items():
    teacher_lessons = [
        var for var in LESSON_VARIABLES
        if get_lesson_info(var)[1] in courses
    ]

    # For every pair of lessons taught by the same teacher
    for i in range(len(teacher_lessons)):
        for j in range(i + 1, len(teacher_lessons)):
            var_a = teacher_lessons[i]
            var_b = teacher_lessons[j]

            # Constraint: Block of Lesson A != Block of Lesson B
            problem.addConstraint(lambda a, b: a[0] != b[0], (var_a, var_b))

# Constraint C: Room Conflicts (A physical room can only host one lesson per block)
# All lessons assigned to a physical room must be in unique (Block) combinations.

# Create a list of all lesson pairs (total of 30 variables, 435 pairs)
all_lesson_pairs = []
for i in range(len(LESSON_VARIABLES)):
    for j in range(i + 1, len(LESSON_VARIABLES)):
        all_lesson_pairs.append((LESSON_VARIABLES[i], LESSON_VARIABLES[j]))

# Constraint: If two lessons use a *physical* room, they must not be in the same block.
# We exclude 'Online' room from this check.
PHYSICAL_ROOMS = [r for r in ROOMS if r != 'Online']

def room_overlap_constraint(a, b):
    # a and b are (Block, Room) tuples
    block_a, room_a = a
    block_b, room_b = b

    # If they are in the same block AND
    if block_a == block_b:
        # If they are in the same PHYSICAL room, it's a conflict (False)
        if room_a == room_b and room_a in PHYSICAL_ROOMS:
            return False
        # If one is Online, they can use the same block as a physical room (True)
    return True # Blocks are different OR rooms are different OR rooms are both online

for var_a, var_b in all_lesson_pairs:
    problem.addConstraint(room_overlap_constraint, (var_a, var_b))

# Soft Constraint Implementation (Distinct Days) - Implemented as Hard Constraint for demo

def get_day(block):
    # Blocks 1-4 = Mon, 5-8 = Tue, etc.
    return DAYS[(block - 1) // 4]

def distinct_days_constraint(a, b):
    # a and b are (Block, Room) tuples
    day_a = get_day(a[0])
    day_b = get_day(b[0])
    return day_a != day_b

# For every pair of lessons of the SAME COURSE, they must be on distinct days.
for class_id in CLASS_COURSES.keys():
    for course in CLASS_COURSES[class_id]:
        var_a = f'{class_id}_{course}_L1'
        var_b = f'{class_id}_{course}_L2'
        problem.addConstraint(distinct_days_constraint, (var_a, var_b))


# --- 6. SOLVE AND DISPLAY RESULTS ---

print(f"Total Lesson Variables to schedule: {len(LESSON_VARIABLES)}")
print("Attempting to find a feasible timetable satisfying Hard Constraints...")

# Finding the first solution
solutions = problem.getSolution()

if solutions:
    print("\n✅ Found a Feasible Timetable Solution (Hard Constraints Met).\n")

    # Structure the result for display
    timetable_data = {}
    for var, (block, room) in solutions.items():
        class_id, course, lesson_index, teacher = get_lesson_info(var)
        day = get_day(block)

        if class_id not in timetable_data:
            timetable_data[class_id] = {}
        if day not in timetable_data[class_id]:
            timetable_data[class_id][day] = {}

        timetable_data[class_id][block] = {
            'Course': course,
            'Teacher': teacher,
            'Room': room
        }

    # --- Display Timetable ---

    BLOCK_HEADERS = [f'B{i} ({DAYS[(i-1)//4]})' for i in range(1, 21)]

    print("-" * 150)
    print(f"{'CLASS':<5} | {'Block 1-4 (Mon)':<48} | {'Block 5-8 (Tue)':<48} | ...")
    print("-" * 150)

    for class_id in sorted(timetable_data.keys()):
        row = f"{class_id:<5} | "

        # Populate the time blocks for the class
        daily_schedule = []
        for block in BLOCKS:
            lesson_info = timetable_data[class_id].get(block)

            if lesson_info:
                content = f"{lesson_info['Course']}/{lesson_info['Room']}"
            else:
                content = '---'

            daily_schedule.append(f"{content:<12}")

        # Join and format the row
        row += ' | '.join(daily_schedule)
        print(row)

    print("-" * 150)

else:
    print("\n❌ No solution found that satisfies all hard and distinct-day constraints with the given resources.")

  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24061 sha256=8a43c1e95d7db0cb822dc13447c10f9ee5b5d48080f81dcc07cfcb52db932b27
  Stored in directory: /root/.cache/pip/wheels/c1/d2/3d/082849b61a9c6de02d4a7c8a402c224640f08d8a971307b92b
Successfully built python-constraint
Total Lesson Variables to schedule: 30
Attempting to find a feasible timetable satisfying Hard Constraints...

✅ Found a Feasible Timetable Solution (Hard Constraints Met).

------------------------------------------------------------------------------------------------------------------------------------------------------
CLASS | Block 1-4 (Mon)                                  | Block 5-8 (Tue)                                  | ...
------------------------------------------------------------------------------------------------------------------------------------------------------
t01   | ---          | ---          | ---        